# Tratativa dos dados para as reviews 3 e 4

## Gerando modelo, treinando e fazendo a predição

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 66.7 MB/s 
     |████████████████████████████████| 895 kB 44.5 MB/s 
     |████████████████████████████████| 596 kB 80.0 MB/s 
     |████████████████████████████████| 61 kB 605 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification, InputExample, InputFeatures
import tensorflow as tf
import spacy
import numpy as np

In [ ]:
# Lendo os datasets
df_reviews = pd.read_csv('drive/MyDrive/Colab Notebooks/TCC/default_dataset.csv')
df_reviews_to_label = pd.read_csv('drive/MyDrive/Colab Notebooks/TCC/dataset_without_labels.csv')

In [ ]:
# Verificando proporção e classes
classlabel_count = df_reviews.label.value_counts()
print('Class 0:', classlabel_count[0])
print('Class 1:', classlabel_count[1])
print('Proportion:', round(classlabel_count[0] / classlabel_count[1], 2), ': 1')

Class 0: 3959
Class 1: 28202
Proportion: 0.14 : 1


In [ ]:
# Inicializando o modelo BERT
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", max_length=512, padding='longest')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1)

In [ ]:
"""Extração de samples para serem usadas no treino do algoritmo"""
samples_balanced_reviews = pd.DataFrame()
labels = df_reviews["label"].unique()

for each_label in labels:
    df_this_rate = df_reviews[df_reviews["label"] == each_label].sample(n=1500)
    samples_balanced_reviews = samples_balanced_reviews.append(df_this_rate)

samples_balanced_reviews = samples_balanced_reviews.sample(frac=1)

samples_balanced_reviews_validation = pd.DataFrame()
labels = df_reviews["label"].unique()

for each_label in labels:
    df_this_rate = df_reviews[df_reviews["label"] == each_label].sample(n=500)
    samples_balanced_reviews_validation = samples_balanced_reviews_validation.append(df_this_rate)

samples_balanced_reviews_validation = samples_balanced_reviews_validation.sample(frac=1)

print(len(samples_balanced_reviews.index))
print(len(samples_balanced_reviews_validation.index))

3000
1000


In [ ]:
"""Setando as labels para 0, por motivos de erros no generate do tf, que não aceita mais de um tipo."""
df_reviews_to_label["label"] = 0

In [ ]:
"""Funções para converter os dados para serem utilizados no modelo do tensorflow"""

def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
    train_InputExamples = train.apply(lambda x: InputExample(guid=None,
                                                              text_a = x[DATA_COLUMN], 
                                                              text_b = None,
                                                              label = x[LABEL_COLUMN]), axis = 1)

    validation_InputExamples = test.apply(lambda x: InputExample(guid=None,
                                                              text_a = x[DATA_COLUMN], 
                                                              text_b = None,
                                                              label = x[LABEL_COLUMN]), axis = 1)

    return train_InputExamples, validation_InputExamples

train_InputExamples, validation_InputExamples = convert_data_to_examples(samples_balanced_reviews, samples_balanced_reviews_validation, 'review_text', 'label')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=512):
    features = []

    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=True,
            return_attention_mask=True,
            padding='max_length',
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'review_text'
LABEL_COLUMN = 'label'

In [ ]:
"""Gerando os batchs de tamanho 8 para serem processados no modelo"""

train_InputExamples, validation_InputExamples = convert_data_to_examples(samples_balanced_reviews, samples_balanced_reviews_validation, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)

train_data = train_data.shuffle(100).batch(8).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(8)

In [ ]:
"""Fine-tuning do BERT para os dados de sentimentos e verificando a perfomrnace.
Acurácia na validação ficou acima de 92 % nas 3 vezes que o algoritmo rodou com 
amostras diferentes"""

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
750/750 [==============================] - 4224s 6s/step - loss: 0.1344 - accuracy: 0.9490 - val_loss: 0.1186 - val_accuracy: 0.9680
Epoch 2/2
750/750 [==============================] - 4273s 6s/step - loss: 0.0156 - accuracy: 0.9965 - val_loss: 0.1532 - val_accuracy: 0.9690


In [ ]:
"""Aplicando predições nos dados de ratings 3 e 4, após o modelo estar refinado 
nas predições do que é uma avaliação negativa e positiva"""

train_InputExamples, validation_InputExamples = convert_data_to_examples(samples_balanced_reviews, df_reviews_to_label, DATA_COLUMN, LABEL_COLUMN)
validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(8)

tf_predictions = model.predict(validation_data)
pred_labels = np.argmax(tf_predictions.logits, axis=1)
df_reviews_to_label["label"] = pred_labels

## Tratando os dados já com as labels

In [ ]:
"""Carregando linguagem do spacy inglês para ser utilizado no processo de 
tokenização"""
nlp_lemma = spacy.load('en_core_web_sm')


def convert_text_to_array(text):
    """Converte o texto para uma array utilizando o spacy"""
    text_converted = nlp_lemma(text)
    
    return [each_word for each_word in text_converted]


def get_token_and_lemma(text):
    """Gera os lemmas das palavras"""
    return [each_word.lemma_ for each_word in text]


def remove_stop_words(tokens):
    """Remove stop words, possíveis citações de usuários com @ e 
       mantém apenas os lemmas alfabéticos (remove pontuações)"""
    return [each_word for each_word in tokens \
            if not nlp_lemma.vocab[each_word].is_stop \
            and nlp_lemma.vocab[each_word].is_alpha \
            and each_word[:1] != "@"]

In [ ]:
"""Fazendo a mesma limpeza e tokenização realizada nos dados das reviews já
classificadas"""
df_reviews_to_label['tokens'] = df_reviews_to_label['review_text'].apply(lambda x: convert_text_to_array(x))
df_reviews_to_label['tokens'] = df_reviews_to_label['tokens'].apply(lambda x: get_token_and_lemma(x))
df_reviews_to_label['tokens'] = df_reviews_to_label['tokens'].apply(lambda x: remove_stop_words(x))

In [ ]:
df_reviews_to_label.to_csv('drive/My Drive/Colab Notebooks/TCC/without_label_labeled_bert.csv')

In [ ]:
df_reviews_to_label.shape

(18743, 11)